In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV2,EfficientNetB0,EfficientNetB4,Xception
import glob
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train = pd.read_csv("../input/insat3d-infrared-raw-cyclone-images-20132021/insat_3d_ds - Sheet.csv")

In [4]:
train.head()

,img_name,label
0,25.jpg,25
1,27.jpg,27
2,28.jpg,28
3,30.jpg,30
4,30(1).jpg,30


In [5]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [6]:
train_data = train_datagen.flow_from_dataframe(train,directory="/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED",subset="training",
                                               x_col="img_name",y_col="label",target_size=(512, 512),batch_size=16,class_mode='raw')

Found 136 validated image filenames.


In [7]:
def build_model():
    base = Xception(weights="imagenet", include_top=False, input_tensor=Input(shape=(512, 512, 3)))

    base.trainable = False

    flatten = base.output
    flatten = Flatten()(flatten)

    bboxHead = Dense(64, activation="relu")(flatten)
    bboxHead = Dense(32, activation="relu")(bboxHead)
    bboxHead = Dense(1, activation="linear")(bboxHead)

    model = Model(inputs=base.input, outputs=bboxHead)
    
    return model

In [8]:
model = build_model()

In [9]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])

save_best = tf.keras.callbacks.ModelCheckpoint("Model.h5", monitor='loss',save_best_only=True, verbose=1)

In [10]:
model.fit(train_data, epochs=50, callbacks=[save_best])

Epoch 1/50
9/9 [==============================] - ETA: 0s - loss: 31.6781 - root_mean_squared_error: 41.2240
Epoch 1: loss improved from inf to 31.67808, saving model to Model.h5
9/9 [==============================] - 20s 650ms/step - loss: 31.6781 - root_mean_squared_error: 41.2240
Epoch 2/50
9/9 [==============================] - ETA: 0s - loss: 16.4000 - root_mean_squared_error: 20.9655
Epoch 2: loss improved from 31.67808 to 16.39997, saving model to Model.h5
9/9 [==============================] - 4s 493ms/step - loss: 16.4000 - root_mean_squared_error: 20.9655
Epoch 3/50
9/9 [==============================] - ETA: 0s - loss: 12.0815 - root_mean_squared_error: 15.3742
Epoch 3: loss improved from 16.39997 to 12.08154, saving model to Model.h5
9/9 [==============================] - 5s 567ms/step - loss: 12.0815 - root_mean_squared_error: 15.3742
Epoch 4/50
9/9 [==============================] - ETA: 0s - loss: 9.0269 - root_mean_squared_error: 10.9402
Epoch 4: loss improved from 12.0